In [1]:
from matplotlib import pyplot
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [4]:
#Parameters
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

#Mesh
x  = numpy.linspace(xmin,xmax,nx)
y  = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

#Initial conditions
psi = numpy.zeros((ny, nx)) #Create a XxY vector of zeros for psi
w = numpy.zeros((ny, nx)) #Create a XxY vector of zeros for omega (w)


NameError: name 'xmin' is not defined

In [3]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [ ]:
def stokes(w, psi, dx, dy, u, l1_target):
    